In [ ]:
# --- Para el proceso de datos y visualización ---
#version final: 1.0.0
import numpy as np
import pandas as pd
import json 
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'
from google.cloud import storage



# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos al gener
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 
from vertexai.vision_models import Image

# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.
from vertexai.vision_models import Video, VideoSegmentConfig

In [ ]:
#Fragmentación de video local con FFmpeg

import os
import subprocess

def fragmentar_video_local_ffmpeg_preciso(
    ruta_video_local: str,
    carpeta_salida: str,
    duracion_segmento: int
):
    """
    Fragmenta un video local en segmentos de X segundos usando ffmpeg,
    forzando la duración exacta de los segmentos, y los guarda en una carpeta local.
    El audio será recodificado y sincronizado con los cortes forzados.
    """
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    nombre_base = os.path.splitext(os.path.basename(ruta_video_local))[0]
    output_pattern = os.path.join(carpeta_salida, f"{nombre_base}_segment_%03d.mkv")

    # Comando de ffmpeg para segmentación precisa
    # -force_key_frames: Asegura que se inserta un keyframe en cada punto de corte deseado.
    # -segment_frames: Forzará el corte basado en el número de frames si fuera necesario,
    #                  pero segment_time con force_key_frames es más efectivo para tiempo.
    cmd = [
        "ffmpeg",
        "-i", ruta_video_local,
        "-c:v", "libx264",                      # Recodifica el video
        "-force_key_frames", f"expr:gte(t,n_forced*{duracion_segmento})", # Fuerza keyframes en cada intervalo
        "-c:a", "aac",                          # Recodifica el audio
        "-map", "0",
        "-f", "segment",
        "-segment_time", str(duracion_segmento),# Duración deseada del segmento
        "-reset_timestamps", "1",               # Reinicia el timestamp para cada segmento
        output_pattern
    ]

    print(f"Fragmentando video con ffmpeg en segmentos de {duracion_segmento} segundos...")
    try:
        subprocess.run(cmd, check=True)
        fragmentos = sorted([
            f for f in os.listdir(carpeta_salida)
            if f.startswith(f"{nombre_base}_segment_") and f.endswith(".mkv")
        ])
        print(f"¡Listo! {len(fragmentos)} fragmentos guardados en: {carpeta_salida}")

    except subprocess.CalledProcessError as e:
        print(f"Error al fragmentar el video: {e}")
        print("Asegúrate de que FFmpeg está instalado y accesible en tu PATH.")

# Ejemplo de uso:
# Asegúrate de que FFmpeg esté instalado y en tu PATH.
# Puedes descargarlo desde https://ffmpeg.org/download.html
fragmentar_video_local_ffmpeg_preciso(
    ruta_video_local="/Users/wivboost/Downloads/mexicosta.mkv",
    #guardar en local
    carpeta_salida="/Users/wivboost/Downloads/fragmentos_mexicosta_precisos", # Nueva carpeta de salida para no sobreescribir
    duracion_segmento=60
)

Fragmentando video con ffmpeg en segmentos de 60 segundos...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

KeyboardInterrupt: 

In [8]:
import os
import subprocess
from google.cloud import storage

def fragmentar_y_subir_video_ffmpeg_tmp(
    ruta_video_local: str,
    gcs_bucket_salida: str,
    carpeta_destino_gcs: str,
    duracion_segmento: int,
    carpeta_tmp: str = "/tmp"
):
    """
    Fragmenta un video local en segmentos de X segundos usando ffmpeg,
    sube los fragmentos directamente a una carpeta específica de un bucket de GCS y los elimina localmente.
    """
    nombre_base = os.path.splitext(os.path.basename(ruta_video_local))[0]
    output_pattern = os.path.join(carpeta_tmp, f"{nombre_base}_%03d.mkv")

    cmd = [
        "ffmpeg",
        "-i", ruta_video_local,
        "-c:v", "libx264",
        "-force_key_frames", f"expr:gte(t,n_forced*{duracion_segmento})",
        "-c:a", "aac",
        "-map", "0",
        "-f", "segment",
        "-segment_time", str(duracion_segmento),
        "-reset_timestamps", "1",
        output_pattern
    ]

    print(f"Fragmentando video con ffmpeg en segmentos de {duracion_segmento} segundos...")
    try:
        subprocess.run(cmd, check=True)
        fragmentos = sorted([
            f for f in os.listdir(carpeta_tmp)
            if f.startswith(f"{nombre_base}_") and f.endswith(".mkv")
        ])
        print(f"¡Listo! {len(fragmentos)} fragmentos generados en {carpeta_tmp}")
    except subprocess.CalledProcessError as e:
        print(f"Error al fragmentar el video: {e}")
        return

    # Subir los fragmentos al bucket de salida en la carpeta deseada
    storage_client = storage.Client()
    bucket_out = storage_client.bucket(gcs_bucket_salida)
    print(f"Subiendo {len(fragmentos)} fragmentos a gs://{gcs_bucket_salida}/{carpeta_destino_gcs}/ ...")
    for frag in fragmentos:
        local_path = os.path.join(carpeta_tmp, frag)
        blob_dest = f"{carpeta_destino_gcs}/{frag}"
        bucket_out.blob(blob_dest).upload_from_filename(local_path)
        print(f"  ✅ Subido: gs://{gcs_bucket_salida}/{blob_dest}")
        os.remove(local_path)  # Elimina el fragmento local después de subirlo

    print("¡Listo! Todos los fragmentos subidos y eliminados localmente.")

# Ejemplo de uso:
fragmentar_y_subir_video_ffmpeg_tmp(
    ruta_video_local="/Users/wivboost/Downloads/mexicosta.mkv",
    gcs_bucket_salida="vboxiooof",
    carpeta_destino_gcs="Videos/Prueba",
    duracion_segmento=60
)

Fragmentando video con ffmpeg en segmentos de 60 segundos...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

¡Listo! 129 fragmentos generados en /tmp
Subiendo 129 fragmentos a gs://vboxiooof/Videos/Prueba/ ...
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_000.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_001.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_002.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_003.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_004.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_005.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_006.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_007.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_008.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_009.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_010.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_011.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_012.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_013.mkv
  ✅ Subido: gs://vboxiooof/Videos/Prueba/mexicosta_014.mkv
  ✅ Subido: gs

In [9]:
import os

def limpiar_fragmentos_tmp(nombre_base: str, carpeta_tmp: str = "/tmp"):
    """
    Elimina todos los fragmentos temporales generados por ffmpeg en la carpeta indicada.
    """
    archivos = [
        f for f in os.listdir(carpeta_tmp)
        if f.startswith(f"{nombre_base}_segment_") and f.endswith(".mkv")
    ]
    for f in archivos:
        try:
            os.remove(os.path.join(carpeta_tmp, f))
            print(f"Eliminado: {f}")
        except Exception as e:
            print(f"No se pudo eliminar {f}: {e}")

# Ejemplo de uso:
limpiar_fragmentos_tmp("mexicosta", carpeta_tmp="/tmp")